In [105]:
import xgboost
import joblib
import numpy as np
import pandas as pd

#### Load model, feature names, testing data and counterfactuals from testing set

In [106]:
#model = joblib.load('files/model_asap7_776_counter_both_ori_adjusted')
model = joblib.load('files/model_asap7_776_counter_both_ori_adjusted_clip')
#model = joblib.load('files/model_asap7_776_counter_both_ori_adjusted_taper')
#model = joblib.load('files/model_asap7_776_counter_both_ori_adjusted_late_taper')

feature_names = joblib.load('files/feature_names_776')

data_test_ori = pd.read_csv('ASAP7 Test Set.tsv', sep='\t')

feat_test_ori = joblib.load('files/asap7_features_test_776')

feats_counter_test_776 = joblib.load('files/feats_counter_test_error_776')
np.array(feats_counter_test_776).shape

(100, 31, 1, 776)

#### Load the scores of the original essays

In [107]:
scores_ori = []
row_ctr = 0

for feat in feat_test_ori[:100]:
    dmat = xgboost.DMatrix(feat.reshape(1,-1), feature_names=feature_names)
    score = model.predict(dmat)
    scores_ori.append(score[0])
    row_ctr = row_ctr + 1
    print("Finished row number : ", row_ctr)

print("row number: ", row_ctr)

Finished row number :  1
Finished row number :  2
Finished row number :  3
Finished row number :  4
Finished row number :  5
Finished row number :  6
Finished row number :  7
Finished row number :  8
Finished row number :  9
Finished row number :  10
Finished row number :  11
Finished row number :  12
Finished row number :  13
Finished row number :  14
Finished row number :  15
Finished row number :  16
Finished row number :  17
Finished row number :  18
Finished row number :  19
Finished row number :  20
Finished row number :  21
Finished row number :  22
Finished row number :  23
Finished row number :  24
Finished row number :  25
Finished row number :  26
Finished row number :  27
Finished row number :  28
Finished row number :  29
Finished row number :  30
Finished row number :  31
Finished row number :  32
Finished row number :  33
Finished row number :  34
Finished row number :  35
Finished row number :  36
Finished row number :  37
Finished row number :  38
Finished row number :

In [108]:
#joblib.dump(scores_ori, "scores_ori_test")
#scores_ori = joblib.load("scores_ori_test_776")

#### Predict the scores of the counterfactuals

In [109]:
scores_counter = []

row_ctr = 0

for row in feats_counter_test_776:
    row_score = []
    for essay in row:
        essay = xgboost.DMatrix(essay, feature_names=feature_names)
        score = model.predict(essay)
        row_score.append(score[0])
    scores_counter.append(row_score)
    row_ctr = row_ctr + 1
    print("Finished row number : ", row_ctr)

print("row number: ", row_ctr)

Finished row number :  1
Finished row number :  2
Finished row number :  3
Finished row number :  4
Finished row number :  5
Finished row number :  6
Finished row number :  7
Finished row number :  8
Finished row number :  9
Finished row number :  10
Finished row number :  11
Finished row number :  12
Finished row number :  13
Finished row number :  14
Finished row number :  15
Finished row number :  16
Finished row number :  17
Finished row number :  18
Finished row number :  19
Finished row number :  20
Finished row number :  21
Finished row number :  22
Finished row number :  23
Finished row number :  24
Finished row number :  25
Finished row number :  26
Finished row number :  27
Finished row number :  28
Finished row number :  29
Finished row number :  30
Finished row number :  31
Finished row number :  32
Finished row number :  33
Finished row number :  34
Finished row number :  35
Finished row number :  36
Finished row number :  37
Finished row number :  38
Finished row number :

In [110]:
#joblib.dump(scores_counter, "scores_counterfactual_test_newmodel_both_776")
#scores_counter = joblib.load('scores_counterfactual_test_newmodel_both_776')

In [111]:
np.array(scores_counter).shape

(100, 31)

#### Compare the scores Original vs Counterfactual

In [112]:
comparison = []

ctr = 1

for idx, row_scores in enumerate(scores_counter):
    ori_score = scores_ori[idx]
    compare_row = []
    for score in row_scores:

        if ctr <= 10:
            print(ctr)
            print("ori: ", ori_score)
            print("counter: ", score)
        
        score_diff = abs(score - ori_score)
        if score <= ori_score:
            compare_row.append((True, score_diff, score))
        else:
            compare_row.append((False, score_diff, score))
        ctr = ctr + 1
        
    comparison.append(compare_row)

1
ori:  13.925055
counter:  18.576624
2
ori:  13.925055
counter:  19.044237
3
ori:  13.925055
counter:  20.064869
4
ori:  13.925055
counter:  18.610079
5
ori:  13.925055
counter:  18.329142
6
ori:  13.925055
counter:  16.080814
7
ori:  13.925055
counter:  16.683315
8
ori:  13.925055
counter:  15.919935
9
ori:  13.925055
counter:  14.5403185
10
ori:  13.925055
counter:  17.363937


In [113]:
comparison

[[(False, 4.6515694, 18.576624),
  (False, 5.1191826, 19.044237),
  (False, 6.1398144, 20.064869),
  (False, 4.6850243, 18.610079),
  (False, 4.404087, 18.329142),
  (False, 2.1557598, 16.080814),
  (False, 2.7582607, 16.683315),
  (False, 1.9948807, 15.919935),
  (False, 0.61526394, 14.5403185),
  (False, 3.4388828, 17.363937),
  (False, 2.2781734, 16.203228),
  (False, 1.7613745, 15.686429),
  (False, 2.7273273, 16.652382),
  (False, 1.8575802, 15.782635),
  (False, 1.4643917, 15.389446),
  (False, 1.2838097, 15.208864),
  (False, 0.053759575, 13.978814),
  (True, 0.914443, 13.010612),
  (False, 1.2875605, 15.212615),
  (True, 0.8091593, 13.115895),
  (True, 1.0936117, 12.831443),
  (False, 1.2473106, 15.172365),
  (True, 0.21494675, 13.710108),
  (True, 1.2231627, 12.701892),
  (True, 0.5595684, 13.365486),
  (True, 0.45165253, 13.473402),
  (True, 1.8822937, 12.042761),
  (True, 3.112342, 10.812713),
  (True, 2.0335855, 11.891469),
  (True, 1.9187927, 12.006262),
  (True, 3.617754,

#### Count the number of False and mean of score difference for the 'False' predictions

In [114]:
# Initialize counters and lists to hold False values' second column values
false_count = 0
false_values = []
new_label = []
true_count = 0

# Iterate through each sublist
for idx, sublist in enumerate(comparison):
    ori = scores_ori[idx]
    row_label = []
    for value in sublist:

        formula = ori - value[1]
        
        if value[0] == False:  # If the first element is False
            false_count += 1
            false_values.append(value[1])  # Collect the second element (the difference)
            row_label.append(formula)
        else:
            true_count += 1
            row_label.append(value[2])
    new_label.append(row_label)
    
# Calculate the mean of the differences for False values only
if false_count > 0:
    mean_difference = np.mean(false_values)
else:
    mean_difference = 0  # Handle case where there are no False values

print(f"Count of False values: {false_count}")
print(f"Count of True values: {true_count}")
print(f"Mean of the differences for False values: {mean_difference}")


Count of False values: 576
Count of True values: 2524
Mean of the differences for False values: 2.8887939453125


In [115]:
np.array(comparison).shape

(100, 31, 3)

In [116]:
acc = (true_count / 3100) * 100
acc

81.41935483870968

In [80]:
np.array(new_label).shape

(100, 31)

In [18]:
#joblib.dump(new_label, 'new_label_test')